<a href="https://colab.research.google.com/github/chungocchien/NLPtoturial/blob/main/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers datasets
!pip install evaluate
!pip install rouge_score

In [ ]:
from transformers import pipeline
import evaluate
rouge = evaluate.load('rouge')

In [ ]:
!pip install gdown
import gdown
# Download vietnamese syllable file
train_url_path = 'https://drive.google.com/file/d/1sOqpFAbnAPa4G_zzCpIcWKU-rFlpANpi/view?usp=sharing'
train_filename = 'train_articles.json'
gdown.download(train_url_path, train_filename, quiet=False,fuzzy=True)

# Download Vietnamese corpus file
val_url_path = 'https://drive.google.com/file/d/1yJJ5F_MndddW236Yl50cOZ2dJAmoTX_e/view?usp=sharing'
val_filename = 'val_articles.json'
gdown.download(val_url_path, val_filename, quiet=False,fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1sOqpFAbnAPa4G_zzCpIcWKU-rFlpANpi
To: /content/train_articles.json
100%|██████████| 51.4M/51.4M [00:00<00:00, 78.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yJJ5F_MndddW236Yl50cOZ2dJAmoTX_e
To: /content/val_articles.json
100%|██████████| 10.2M/10.2M [00:00<00:00, 48.4MB/s]


'val_articles.json'

In [ ]:
## Standard libraries
import os
import numpy as np
import random
import math
import json

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
from torch.nn import CrossEntropyLoss, NLLLoss
from torch.utils.data import DataLoader

## Transformers & Dataset
import datasets
from datasets import Dataset

import transformers
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from transformers import AdamW
from transformers import MT5ForConditionalGeneration
from transformers.modeling_outputs import BaseModelOutput, Seq2SeqLMOutput

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


In [ ]:
max_encoder_length = 320
max_decoder_length = 80

batch_size = 4
learning_rate=5e-4
n_epochs=5

In [ ]:
# Read data
train_data = json.load(open("train_articles.json", encoding="utf-8"))
train_data = Dataset.from_dict(train_data)

val_data = json.load(open("val_articles.json", encoding="utf-8"))
val_data = Dataset.from_dict(val_data)

In [ ]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small", use_fast=False)

def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(batch["messages"], padding="max_length", truncation=True, max_length=max_encoder_length, add_special_tokens=True)
    outputs = tokenizer(batch["snippets"], padding="max_length", truncation=True, max_length=max_decoder_length, add_special_tokens=True)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [ ]:
train_data = train_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=256,
    remove_columns=["titles", "domains"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

val_data = val_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=256,
    remove_columns=["titles", "domains"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Convert dataset to dataloader
train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=4, shuffle=False)

In [ ]:
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small').to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for epoch in range(n_epochs):
    for i, batch in enumerate(train_dataloader):
        batch = {k:v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=batch['input_ids'],
                        attention_mask=batch['attention_mask'],
                        decoder_attention_mask=batch['decoder_attention_mask'],
                        labels=batch['labels'],
                        return_dict=True)

        loss, logits = outputs['loss'], outputs['logits']
        optimizer.zero_grad()
        outputs['loss'].backward()
        optimizer.step()

        if i % 20 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, 5, i, len(train_dataloader), loss.item()))

Epoch [1/5], Step [0/2500], Loss: 12.4980
Epoch [1/5], Step [20/2500], Loss: 2.9970
Epoch [1/5], Step [40/2500], Loss: 2.5958
Epoch [1/5], Step [60/2500], Loss: 2.2779
Epoch [1/5], Step [80/2500], Loss: 2.1578
Epoch [1/5], Step [100/2500], Loss: 2.6812
Epoch [1/5], Step [120/2500], Loss: 1.8482
Epoch [1/5], Step [140/2500], Loss: 2.4422
Epoch [1/5], Step [160/2500], Loss: 2.5129
Epoch [1/5], Step [180/2500], Loss: 2.0834
Epoch [1/5], Step [200/2500], Loss: 2.1669
Epoch [1/5], Step [220/2500], Loss: 2.3093
Epoch [1/5], Step [240/2500], Loss: 2.0247
Epoch [1/5], Step [260/2500], Loss: 1.4552
Epoch [1/5], Step [280/2500], Loss: 1.5551
Epoch [1/5], Step [300/2500], Loss: 1.3602
Epoch [1/5], Step [320/2500], Loss: 1.8841
Epoch [1/5], Step [340/2500], Loss: 1.7325
Epoch [1/5], Step [360/2500], Loss: 1.6294
Epoch [1/5], Step [380/2500], Loss: 1.8089
Epoch [1/5], Step [400/2500], Loss: 1.8329
Epoch [1/5], Step [420/2500], Loss: 1.2953
Epoch [1/5], Step [440/2500], Loss: 1.4877
Epoch [1/5], Ste

In [ ]:
# Generate summary of an article
def summarize(text, min_length=20):
    inputs = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    outputs = model.generate(input_ids, attention_mask=attention_mask, min_length=min_length)

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
s = """
Ngày 27/3 , Cơ quan Cảnh sát điều tra Công an TP. Hưng Yên , tỉnh Hưng Yên cho biết , đơn vị vừa ra quyết định khởi tố vụ án , khởi tố bị can đối với đối tượng Mai Văn Thương ( SN 1989 , trú tại đội 11 , thôn An Chiểu 1 , xã Liên Phương , TP. Hưng Yên ) để điều tra về hành vi trộm cắp tài sản . Theo tài liệu điều tra của cơ quan công an , vào khoảng 7h30 ngày 13/3 , lợi dụng gia đình ông Mai Văn Thịnh ( chú ruột đối tượng Thương ) ở cạnh nhà đi vắng , đối tượng này đã đạp gãy chấn song cửa sổ , đột nhập vào nhà ông Thịnh trộm cắp 121kg thóc mang bán cho người cùng thôn lấy 700.000 đ . Không dừng lại , sau đó đối tượng tiếp tục quay lại lục soát tủ nhà ông Thịnh trộm cắp 8.500.000 đ tiền mặt ( ông Thịnh để dưới đáy tủ ) , rồi dùng số tiền trên để đi mua ma tuý về sử dụng và tiêu xài hết 6.080.000 đ . Đến ngày 15/3 , đối tượng Thương đã đến Cơ quan điều tra Công an TP. Hưng Yên tự thú và khai nhận toàn bộ hành vi phạm tội của mình , đồng thời giao nộp cho cơ quan công an 3.120.000 đ . Hiện Công an TP. Hưng Yên đã thu giữ toàn bộ 121kg thóc đối tượng đã trộm cắp để trao trả cho gia đình ông Thịnh . Được biết Thương là đối tượng nghiện ma tuý từ nhiều năm nay , đã có 1 tiền án về tội Tàng trữ trái phép chất ma tuý bị TAND tỉnh Hưng Yên xử phạt 2 năm 3 tháng tù giam . Ra tù năm 2016 , đối tượng này tiếp tục có hành vi cố ý gây thương tích , bị Công an TP. Hưng Yên ra quyết định xử phạt 2,5 triệu đồng . Vụ án đang được Công an TP. Hưng Yên hoàn thiện hồ sơ để xử lý Mai Văn Thương theo quy định của pháp luật .
"""

s_summarize = summarize(s)
s_summarize

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Trong quá trình đạp gãy chấn, đối']

In [ ]:
summarizer = pipeline("summarization", model="VietAI/vit5-large-vietnews-summarization", device=0)
text = """Ngày 6/4, Cơ quan cảnh sát điều tra Công an TPHCM đã có kết luận điều tra, đề nghị VKS cùng cấp truy tố bị can Nguyễn Phương Hằng (52 tuổi, Tổng giám đốc công ty cổ phần Đại Nam), Đặng Anh Quân (tiến sĩ luật) và 3 đồng phạm về tội Lợi dụng các quyền tự do dân chủ xâm phạm lợi ích của Nhà nước, quyền, lợi ích hợp pháp của tổ chức, cá nhân. Đề nghị truy tố bà Nguyễn Phương Hằng, ông Đặng Anh Quân và 3 đồng phạm - 1 Nguyễn Phương Hằng tiếp tục bị đề nghị truy tố (Ảnh: A.X). Theo hồ sơ vụ án, bà Nguyễn Phương Hằng đã lợi dụng, sử dụng 12 kênh mạng xã hội livestream xâm phạm bí mật đời tư, ảnh hưởng uy tín, danh dự nhiều cá nhân như ông Võ Nguyễn Hoài Linh (nghệ sĩ Hoài Linh), bà Nguyễn Thị Mỹ Oanh (ca sĩ Vy Oanh), bà Ðặng Thị Hàn Ni (nhà báo, thạc sĩ luật Hàn Ni), ông Huỳnh Minh Hưng (ca sĩ Ðàm Vĩnh Hưng), bà Trần Thị Thủy Tiên (ca sĩ Thủy Tiên) cùng chồng là Lê Công Vinh, ông Nguyễn Đức Hiển (Phó tổng biên tập báo Pháp luật TPHCM)… Quá trình điều tra, bị can Nguyễn Phương Hằng khai các thông tin phát ngôn khi livestream và đăng tải trên Facebook về các cá nhân trên đều do bà này đọc trên mạng chưa được kiểm chứng và không có cơ sở chứng minh. Tháng 3/2022, bà Nguyễn Phương Hằng bị khởi tố. Mở rộng điều tra vụ án cuối năm ngoái, nhà chức trách đã khởi tố bị can Lê Thị Thu Hà (31 tuổi), Nguyễn Thị Mai Nhi (40 tuổi) và Huỳnh Công Tân (29 tuổi). Công an TPHCM xác định từ tháng 4/2021, bị can Hằng chỉ đạo Nguyễn Thị Mai Nhi tạo lập nhiều tài khoản mạng xã hội TikTok và tạo tài khoản Facebook để đăng thông báo lịch phát sóng các buổi livestream và đăng các bài viết theo yêu cầu của bị can Hằng. Từ tháng 3/2021, bà Lê Thị Thu Hà tham gia sắp xếp góc máy quay, sân khấu để hỗ trợ bị can Hằng livestream. Ðồng thời, Hà còn lập tài khoản Facebook để thông báo lịch phát sóng các buổi livestream và đăng tải bài viết theo yêu cầu của bị can Hằng. Theo chỉ đạo của bị can Hằng, Huỳnh Công Tân sử dụng máy tính, máy quay của Công ty cổ phần Ðại Nam để livestream các buổi nói chuyện của bị can Hằng và phát trực tiếp qua YouTube. Ngoài ra, Tân làm nhiệm vụ đọc lại các bình luận, dẫn chương trình cho các buổi livestream của bị can Nguyễn Phương Hằng. Ðến ngày 30/1/2022, Cơ quan cảnh sát điều tra Công an TPHCM ra quyết định khởi tố bị can đối với Lê Thị Thu Hà, Nguyễn Thị Mai Nhi, Huỳnh Công Tân về tội Lợi dụng các quyền tự do dân chủ xâm phạm lợi ích của Nhà nước, quyền, lợi ích hợp pháp của tổ chức, cá nhân."""

reference = "Cơ quan cảnh sát điều tra xác định bị can Quân nhiều lần phát ngôn, bình luận, cùng tương tác với bị can Hằng về những nội dung xúc phạm danh dự, uy tín, nhân phẩm của nhiều cá nhân."
summary = summarizer(text, min_length=60)
print(summary)

[{'summary_text': 'Bà Nguyễn Phương Hằng lợi dụng 12 kênh mạng xã hội livestream xâm phạm bí mật đời tư của nhiều cá nhân như Hoài Linh , ca sĩ Vy Oanh ... và ảnh hưởng uy tín của các nghệ sĩ trên địa bàn TP HCM , đồng thời chỉ đạo nhân viên tạo tài khoản Facebook để hỗ trợ bị can Hằng livestream .'}]


In [ ]:
train_data['snippets'][0]

'Với bản tính ham chơi , lười làm , có nhiều tiền án tiền sự , lại nghiện ma tuý , Thương đã đột nhập vào nhà chú ruột để trộm hơn 1 tạ thóc và hơn 8 triệu đồng mang đi tiêu xài .'